### Gapminder.org, que recopila estadísticas globales sobre salud, economía y desarrollo humano.

 Columna   | Descripción                                      
 --------- | ------------------------------------------------
 country   | Nombre del país                                  
 continent | Continente al que pertenece el país              
 year      | Año de observación (de 1952 a 2007, cada 5 años)
 lifeExp   | Expectativa de vida al nacer                     
 pop       | Población total                                  
 gdpPercap | Producto Interno Bruto per cápita                
 iso_alpha | Código ISO (3 letras) del país                   
 iso_num   | Código numérico ISO del país                     

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df = px.data.gapminder()
df.head()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4


### Parte 1: Gráficos interactivos básicos

In [ ]:
# Gráfico de barras interactivo
fig = px.bar(df, x="continent", y="pop", title="Población por continente (2007)")
fig.show()

In [ ]:
#Para no ver la barritas por cada país

# Agrupar por continente
df_grouped = df.groupby("continent", as_index=False)["pop"].sum()

# Gráfico de barras por continente
fig = px.bar(
    df_grouped,
    x="continent",
    y="pop",
    title="Población total por continente (2007)",
    text="pop"
)

# Etiquetas con separador de miles y posicionadas encima
fig.update_traces(
    texttemplate='%{text:,}',  # separador de miles
    textposition='outside'
)

# Eje Y con separador también
fig.update_layout(
    yaxis_title="Población",
    xaxis_title="Continente",
    yaxis_tickformat=",",  # formato con separador
)

fig.update_layout(
    height=500,  # Aumenta la altura del gráfico
    yaxis_title="Población",
    xaxis_title="Continente",
    yaxis_tickformat=","
)

fig.show()


In [ ]:
#Ahora para ver barras por cada país, en cada continente

df_pais = df[df["year"] == 2007].copy()

# Ordenar por población (de mayor a menor) dentro de cada continente
df_pais.sort_values(by=["continent", "pop"], ascending=[True, False], inplace=True)

# Definir orden personalizado para los países
country_order = df_pais["country"].tolist()

# Gráfico
fig = px.bar(
    df_pais,
    x="pop",
    y="country",
    color="continent",
    orientation='h',
    hover_name="country",
    title="Población por país en 2007 (ordenado dentro de cada continente)",
    category_orders={"country": country_order},
    width=1000,
    height=700  # Aumentá este valor si hay muchos países
)

fig.update_layout(yaxis={'categoryorder':'array', 'categoryarray':country_order})
fig.show()

In [ ]:
#Gráfico de líneas para ver la esperanza de vida en Uruguay
df_line = px.data.gapminder().query("country == 'Uruguay'")
fig = px.line(df_line, x="year", y="lifeExp", title="Esperanza de vida en Uruguay")
fig.show()

In [ ]:
#Gráfico de líneas para ver la esperanza de vida en varios países
# Filtrar por los países deseados
df_line = df.query("country in ['Uruguay', 'Argentina']")

# Gráfico de líneas
fig = px.line(
    df_line,
    x="year",
    y="lifeExp",
    color="country",
    title="Esperanza de vida: Uruguay vs Argentina"
)

fig.show()


### Parte 2: Gráfico de dispersión

In [ ]:
# Gráfico de dispersión con tamaño y color

fig = px.scatter(
    df, x="gdpPercap", y="lifeExp", animation_frame="year", size="pop", color="continent",
    hover_name="country", log_x=True, size_max=60,
    title="Esperanza de vida vs. PBI per cápita (2007)",
    width=800,
    height=600  # Aumentá este valor si hay muchos países
)
fig.show()
#fig.show("browser")

¿Qué se puede observar?
- Tendencia general: los países con mayor PBI per cápita suelen tener mayor esperanza de vida.
- Asia: se ven burbujas grandes (ej. China, India) desplazándose hacia la derecha con el tiempo (crecimiento económico).
- África: muchos países permanecen con bajo PBI y esperanza de vida, aunque algunos mejoran.
- Europa y Américas: se agrupan en la parte superior derecha (alta riqueza y salud).

In [ ]:
# Filtrar solo los países que quieras comparar
paises = ['Uruguay', 'Argentina', 'Chile']
df_filtrado = df[df['country'].isin(paises)]

# Gráfico de burbujas animado
fig = px.scatter(
    df_filtrado,
    x="gdpPercap",  # Eje X: PBI per cápita
    y="lifeExp",    # Eje Y: Esperanza de vida
    animation_frame="year",  # Animación por año
    animation_group="country",
    size="pop",     # Tamaño de burbuja: población
    color="country",
    hover_name="country",
    log_x=True,
    size_max=60,
    title="Esperanza de vida vs PBI per cápita en América del Sur (1952-2007)",
        width=800,
    height=600  # Aumentá este valor si hay muchos países
)

fig.show()

### Parte 3: Gráfico sunburst (jerárquico)

In [ ]:
#Gráfico sunburst (jerárquico)
fig = px.sunburst(
    df,
    path=["continent", "country"],
    values="pop",
    title="Distribución poblacional",
    width=700,     # ancho en píxeles
    height=500     # alto en píxeles
)
fig.show()


### Parte 4: Mapas

#### Mapa coroplético

In [ ]:
#Mapa coroplético

fig = px.choropleth(
    df,
    locations="iso_alpha",          # Códigos ISO del país
    color="lifeExp",                # Variable a visualizar
    hover_name="country",           # Nombre al pasar el mouse
    color_continuous_scale="Viridis",
    title="Esperanza de vida por país en 2007",
    width=1000,
    height=700  # Aumentá este valor si hay muchos países
)

fig.show()

#### Mapa con burbujas

In [ ]:
#Mapa con burbujas

# Crear mapa de burbujas
fig = px.scatter_geo(
    df,
    locations="iso_alpha",       # Código ISO del país
    color="continent",           # Color por continente
    size="pop",                  # Tamaño de burbuja por población
    hover_name="country",        # Nombre que aparece al pasar el mouse
    projection="natural earth",  # Tipo de proyección del mapa
    title="Población mundial (2007) representada con burbujas",
    width=1000,
    height=700  # Aumentá este valor si hay muchos países
)

fig.show()

#### Mapa con burbujas animado

In [ ]:
#Mapa con burbujas animado

# Crear gráfico
fig = px.scatter_geo(
    df,
    locations="iso_alpha",          # Códigos de país para geolocalización
    color="continent",              # Color por continente
    size="pop",                     # Tamaño de la burbuja por población
    hover_name="country",           # Tooltip con nombre del país
    animation_frame="year",         # Año como fotograma para animación
    projection="natural earth",     # Tipo de proyección del mapa
    title="Evolución de la población mundial (1952–2007)",
    size_max=50,                     # Tamaño máximo de las burbujas
    width=800,
    height=500  # Aumentá este valor si hay muchos países
)

fig.show()


#### Mapa con puntos georeferenciados

##### Para este tipo de mapa vamos a usar otro dataset, ya que Gapminder no tiene los datos con lat y lon

Dataset: `px.data.carshare()`. Contiene información sobre la demanda de autos compartidos (carsharing), por hora y ubicación geográfica.

Variables principales:
* `centroid_lat`, `centroid_lon`: coordenadas de ubicación
* `peak_hour`: hora del día
* `car_hours`: cantidad de horas de autos alquilados en ese punto

In [ ]:
# Cargar dataset
df = px.data.carshare()

# Agrupar por ubicación (lat/lon) y calcular la media de uso
df_zonas = df.groupby(['centroid_lat', 'centroid_lon'], as_index=False)['car_hours'].mean()

# Crear mapa con burbujas proporcionales al uso promedio
fig = px.scatter_map(
    df_zonas,
    lat="centroid_lat",
    lon="centroid_lon",
    size="car_hours",
    color="car_hours",
    size_max=30,
    zoom=10,
    color_continuous_scale="Plasma",
    title="Uso promedio diario de carsharing por ubicación"
)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.show()

#El tamaño y color de cada burbuja indican cuántas horas promedio se usa el servicio allí.